# LangChain + GitHub Models + Azure OpenAI

LangChain is a modular framework designed to streamline the development and integration of applications utilizing large language models (LLMs) by providing tools for effective prompt engineering, data handling, and complex workflows.

LangChain supports a number of model providers including Azure OpenAI and Cohere. Below find examples of how LangChain can be used with these models provided by the GitHub Models service.


## 1. Install dependencies

In [ ]:
%pip install langchain-openai
%pip install langchain-community
%pip install faiss-cpu

## 2. Constructing LangChain's ChatOpenAI class

Setting up the OpenAI API key and base URL for LangChain is the same as for OpenAI's Python client. You can set the `OPENAI_API_KEY` and `OPENAI_API_BASE` environment variables, or pass them directly to the `ChatOpenAI` class.


In [ ]:
from langchain_openai import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv()

if not os.getenv("GITHUB_TOKEN"):
    raise ValueError("GITHUB_TOKEN is not set")

os.environ["OPENAI_API_KEY"] = os.getenv("GITHUB_TOKEN")
os.environ["OPENAI_BASE_URL"] = "https://models.inference.ai.azure.com/"

GPT_MODEL = "gpt-4o-mini"

llm = ChatOpenAI(model=GPT_MODEL)

Now you can use the ChatOpenAI class to interact with the OpenAI API. 

In [ ]:
from IPython.display import Markdown, display

response = llm.invoke("What is the meaning of life?")
display(Markdown(response.content))

## 3. Use a prompt template

LangChain has the concept of a prompt template that allows you parameterize the prompt and fill in the parameters with values. 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a fun assistant that uses a lot of emojis."),
    ("user", "{input}")
])

Next, we combine the promt with the ChatOpenAI into a chain:

In [ ]:
chain = prompt | llm 

Then we invoke the chain with the input data:

In [ ]:
chain.invoke({"input": "when was the first ESC held?"})


## 4. Add a parsing step to your chain

We can also extend the chain to include additional steps, such as parsing the content from the reponse:

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser

chain.invoke({"input": "when was the first ESC held?"})

## 5. Use retrieval to ground the model

Now let's ask for some more recent information:

In [ ]:
chain.invoke({"input": f"Who won the 2024 ESC?"})

Looks like the model is not up to date with the latest contest. Let's help it out with some content from the wikipedia:

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Eurovision_Song_Contest_2024")

docs = loader.load()
len(docs[0].page_content)

Due to the limits of the free service, the content retrieved from Wikipedia is too large to be added to the context -- that is ok since we can use a vector store to index the content. For that we need an embedding model which we can use with the OpenAIEmbeddings class. Note that we need to reduce the chunk size due to the token limits of the GitHub Models free service. 

In [ ]:
from langchain_openai import OpenAIEmbeddings

EMBEDDINGS_MODEL = "text-embedding-3-small"

# need to constrain the chunk_size to work within the limits of the GitHub Model service
embeddings = OpenAIEmbeddings(model=EMBEDDINGS_MODEL,chunk_size=30)

Now, we will use the [FAISS](https://github.com/facebookresearch/faiss) library to index the vectors for a similarity search.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Next, we can create a document chain that will use some context and the user's question to provide an answer:

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""You are a fun assistant that uses a lot of emojis.
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

We could just provide the context directly to the chain like so:

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Who won 2024 ESC?",
    "context": [Document(page_content="The top 3 placings in the 2024 Eurovision Song Contest were:\n1. Switzerland\n2. Croatia\n3. Ukraine")]
})

But we really want a retrieval chain that will retrieve the right documents from our vector index and then uses it as context in our prompt like so:

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

Let's ask this one who won the 2024 ESC:

In [ ]:
response = retrieval_chain.invoke({"input": f"Who won the 2024 ESC?"})
print(response["answer"])


And a few more increasingly detailed questions:

In [ ]:
response = retrieval_chain.invoke({"input": "where did the 2024 ESC take place?"})
print(response["answer"])

In [ ]:
response = retrieval_chain.invoke({"input": "who designed the stage for the 2024 ESC?"})
print(response["answer"])

In [ ]:
response = retrieval_chain.invoke({"input": "when did the semi-finals for the 2024 ESC take place?"})
print(response["answer"])